## UFO Model Deployment

### Goal: Train and deploy model into SageMaker hosting.

Steps:
1. [Load dataset onto Notebook instance memory from S3](#Step-1:-Load-the-data-from-Amazon-S3)
1. [Cleaning, transforming and preparing the dataset](#Step-2:-Cleaning,-transforming-and-preparing-the-dataset)
1. [Training with Linear Learner](#Step-3:-Training-with-Linear-Learner)
1. [Deploying into SageMaker hosting](#Step-4:-Deploying-the-model-into-SageMaker-hosting)

### Imports

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import io
import sagemaker.amazon.common as smac

import boto3
from sagemaker import get_execution_role
import sagemaker

import matplotlib.pyplot as plt
import seaborn as sns

## Step 1: Loading the data from Amazon S3
Read from S3 into Pandas df.

In [ ]:
role = get_execution_role()
bucket='ml_ufo_sightings'
sub_folder = 'ufo_dataset'
data_key = 'ufo_fullset.csv'
data_location = 's3://{}/{}/{}'.format(bucket, sub_folder, data_key)

df = pd.read_csv(data_location, low_memory=False)
df.head()

## Step 2: Cleaning, transforming and preparing the dataset

In [ ]:
# Replace the missing values with the most common shape
df['shape'] = df['shape'].fillna(df['shape'].value_counts().index[0])

# timestamps/dates to 'date' data type
df['reportedTimestamp'] = pd.to_datetime(df['reportedTimestamp'])
df['eventDate'] = pd.to_datetime(df['eventDate'])

# 'Shape' and 'weather' and 'researchoutcome' to 'category' data type.
df['shape'] = df['shape'].astype('category')
df['weather'] = df['weather'].astype('category')
df['researchOutcome'] = df['researchOutcome'].astype('category')

# 'physical evidence' and 'contact' to numerical
df['physicalEvidence'] = df['physicalEvidence'].replace({'Y': 1, 'N': 0})
df['contact'] = df['contact'].replace({'Y': 1, 'N': 0})

# Remove uninformative variables
df.drop(columns=['firstName', 'lastName', 'sighting', 'reportedTimestamp', 'eventDate', 'eventTime'], inplace=True)

# one-hot 'weather' and 'shape' attributes
df = pd.get_dummies(df, columns=['weather', 'shape'])

#Target variable
# Replace the 'researchOutcome' values with 0 = Unexplained, 1 = Explained, 2 = Probable
df['researchOutcome'] = df['researchOutcome'].replace({'unexplained': 0, 'explained': 1, 'probable': 2})

In [ ]:
display(df.head())
display(df.shape)

## Step 3: Training with Linear Learner

In [ ]:
# random shuffle and split
np.random.seed(0)
rand_split = np.random.rand(len(df))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
test_list = rand_split >= 0.9

# train
data_train = df[train_list]
# validation
data_val = df[val_list]
# test
data_test = df[test_list]

# split train into x and y
train_X = data_train.drop(columns='researchOutcome').values
train_y = data_train['researchOutcome'].values
# split val into x and y
val_X = data_val.drop(columns='researchOutcome').values
val_y = data_val['researchOutcome'].values
# split test into x and y
test_X = data_test.drop(columns='researchOutcome').values
test_y = data_test['researchOutcome'].values

Train: Create recordIO file pipe mode training and save on S3.

In [ ]:
train_file = 'ufo_sightings_train_recordIO_protobuf.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, train_X.astype('float32'), train_y.astype('float32'))
f.seek(0)

boto3.Session().resource('s3').Bucket(bucket).Object('implementation_operations_lab/linearlearner_train/{}'.format(train_file)).upload_fileobj(f)
training_recordIO_protobuf_location = 's3://{}/implementation_operations_lab/linearlearner_train/{}'.format(bucket, train_file)
print('The Pipe mode recordIO protobuf training data: {}'.format(training_recordIO_protobuf_location))

Validation: Create recordIO file pipe mode training and save on S3.

In [ ]:
validation_file = 'ufo_sightings_validatioin_recordIO_protobuf.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, val_X.astype('float32'), val_y.astype('float32'))
f.seek(0)

boto3.Session().resource('s3').Bucket(bucket).Object('implementation_operations_lab/linearlearner_validation/{}'.format(validation_file)).upload_fileobj(f)
validate_recordIO_protobuf_location = 's3://{}/implementation_operations_lab/linearlearner_validation/{}'.format(bucket, validation_file)
print('The Pipe mode recordIO protobuf validation data: {}'.format(validate_recordIO_protobuf_location))

Call Linear Learner ECR

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker

container = get_image_uri(boto3.Session().region_name, 'linear-learner', "1")

In [ ]:
# training job name
job_name = 'ufo-linear-learner-job-{}'.format(datetime.now().strftime("%Y%m%d%H%M%S"))
print('Job name {}'.format(job_name))

# Model artifact output path
output_location = 's3://{}/implementation_operations_lab/linearlearner_output'.format(bucket)

Define hyperparameters and train with fit fucnction.

In [ ]:
print('The "feature_dim" hyperparameter is: {}.'.format(data_train.shape[1] - 1))

In [ ]:
sess = sagemaker.Session()

# Setup the LinearLeaner algorithm from the ECR container
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess,
                                       input_mode='Pipe')
# Setup the hyperparameters
linear.set_hyperparameters(feature_dim=22,
                            predictor_type='multiclass_classifier',
                            num_classes=3,
                            early_stopping_patience=3,
                            epochs=15,
                            l1=0.064774153906635,
                            learning_rate=0.0932904024421902,
                            loss='auto',
                            mini_batch_size=744,
                            normalize_data='true',
                            normalize_label='auto',
                            num_models='auto',
                            optimizer='auto',
                            unbias_data='auto',
                            unbias_label='auto',
                            use_bias='true',
                            wd=0.000212481391205101
                          )

# Launch a training job. This method calls the CreateTrainingJob API call
data_channels = {
    'train': training_recordIO_protobuf_location,
    'validation': validate_recordIO_protobuf_location
}
linear.fit(data_channels, job_name=job_name)

In [ ]:
print('Artifact Path: {}/{}/output/model.tar.gz'.format(output_location, job_name))

## Step 4: Deploying the model into SageMaker hosting

Deploy the artifact into SageMaker hosting onto a single m4 instance. This will list in the SageMaker console as "Endpoint".

Next step after this is predicting the test data and evaluating.

In [ ]:
multiclass_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Define function to draw a confusion matrix/heatmap. 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None, 
                          cmap=None):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
            plt.cm.Greens
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Actual',
           xlabel='Predicted')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

### Offline Batch Prediction: Pass 'test_X' into prediction.

In [ ]:
from sagemaker.predictor import json_deserializer, csv_serializer

multiclass_predictor.content_type = 'text/csv'
multiclass_predictor.serializer = csv_serializer
multiclass_predictor.deserializer = json_deserializer

predictions = []
results = multiclass_predictor.predict(test_X)
predictions += [r['predicted_label'] for r in results['predictions']]
predictions = np.array(predictions)

### Call Confusion matrix y_test vs y_pred

In [ ]:
%matplotlib inline
sns.set_context("paper", font_scale=1.4)

y_test = test_y
y_pred = predictions

class_names = np.array(['Unexplained', 'Explained', 'Probable'])

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix',
                      cmap=plt.cm.Blues)
plt.grid(False)
plt.show()

### Test Accuracy, Precision, F1 Scores

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

y_test = data_test['researchOutcome']
y_pred = predictions
scores = precision_recall_fscore_support(y_test, y_pred, average='macro', labels=np.unique(y_pred))
print('Precision is: {}'.format(scores[0]))
print('Recall is: {}'.format(scores[1]))
print('F1 score is: {}'.format(scores[2]))

acc = accuracy_score(y_test, y_pred)
print('Accuracy is: {}'.format(acc))